In [1]:
import duckdb
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

/Users/nrh146/.pyenv/versions/3.12.4/envs/url_clf/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
con = duckdb.connect(':memory:')

In [3]:
MAPPING_PATHS = {
    "huffpo": None,
    "uci": "../../data/uci_categories_attributed.csv",
    "recognasumm": "../../data/recognasumm_categories_attributed.csv"
}

In [14]:
datasets = ["huffpo", "uci", "recognasumm"]
evaluation_metrics = []

# BERT

In [15]:
def evaluate_bert(dataset):
    data = con.execute(f"SELECT * FROM '../../data/processed/{dataset}_test.csv' ").fetch_df()

    if dataset == "recongasumm":
        model_path = f"../../models/bert_multilingual/{dataset}"
    else:
        model_path = f"../../models/bert/{dataset}"

    tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
    model = DistilBertForSequenceClassification.from_pretrained(model_path)
    label_encoder = torch.load(f"{model_path}/label_encoder.pt")

    texts = data['x'].tolist()
    # Tokenize the input
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")

    # Create a DataLoader for batch processing
    ds = TensorDataset(encodings['input_ids'], encodings['attention_mask'])
    dataloader = DataLoader(ds, batch_size=64)  # Adjust batch size as needed

    # Run inference
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(dataloader, total=len(dataloader)):
            input_ids, attention_mask = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_classes = torch.argmax(logits, dim=1)
            predictions.extend(predicted_classes.tolist())

    # Add predictions to the DataFrame
    data['y_pred'] = predictions

    # Encode the labels
    data["y_encoded"] = label_encoder.transform(data["y"])

    # Calculate metrics
    accuracy = accuracy_score(data["y_encoded"], data["y_pred"])
    precision = precision_score(data["y_encoded"], data["y_pred"], average="macro")
    recall = recall_score(data["y_encoded"], data["y_pred"], average="macro")
    f1 = f1_score(data["y_encoded"], data["y_pred"], average="macro")

    return {
        "model": "bert",
        "dataset": dataset,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [16]:
for d in datasets:
    evaluation_metrics.append(evaluate_bert(d))

/var/folders/m8/v1brzj4s56g48wwmpht61td40000gq/T/ipykernel_78306/15403748.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  label_encoder = torch.load(f"{model_path}/labe

# Distant labeling

In [17]:
def evaluate_distant_labeling(dataset):
    mapping = MAPPING_PATHS[dataset]

    if mapping is not None:
        data = con.execute(f"""
        WITH mapping AS (
            SELECT * FROM '{mapping}'
        )
        SELECT x.text AS x, x.y, m.category AS y_pred FROM '../../data/processed/{dataset}_test.csv' x LEFT JOIN mapping m ON SPLIT_PART(SPLIT_PART(x.URL, '://', 2), '/', 2) = m.slug
        """).fetch_df()

        labeled = data[data["y_pred"].notnull()]
        unlabeled = data[data["y_pred"].isnull()]
    else:
        data = con.execute(f"SELECT text AS x, y, NULL AS y_pred FROM '../../data/processed/{dataset}_test.csv' ").fetch_df()
        unlabeled = data
        labeled = None
    
    # load in vectorizer and model
    vectorizer = torch.load(f"../../models/distant/{dataset}/vectorizer.pt")
    model = torch.load(f"../../models/distant/{dataset}/model.pt")
    # vectorize the text
    X = vectorizer.transform(unlabeled["x"])
    # run inference
    y_pred = model.predict(X)
    # add predictions to the DataFrame
    unlabeled["y_pred"] = y_pred
    # combine labeled and unlabeled data
    if labeled is not None:
        labeled = pd.concat([labeled, unlabeled])
    else:   
        labeled = unlabeled
    
    # Calculate metrics
    accuracy = accuracy_score(labeled["y"], labeled["y_pred"])
    precision = precision_score(labeled["y"], labeled["y_pred"], average="macro")
    recall = recall_score(labeled["y"], labeled["y_pred"], average="macro")
    f1 = f1_score(labeled["y"], labeled["y_pred"], average="macro")

    return {
        "model": "distant_labeling",
        "dataset": dataset,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }



In [19]:
for d in datasets:
    evaluation_metrics.append(evaluate_distant_labeling(d))

/var/folders/m8/v1brzj4s56g48wwmpht61td40000gq/T/ipykernel_78306/2992351764.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vectorizer = torch.load(f"../../models/dista

# XGBoost

In [22]:
def evaluate_xgboost(dataset):
    data = con.execute(f"SELECT * FROM '../../data/processed/{dataset}_test.csv' ").fetch_df()

    embedder = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct")

    X = embedder.encode(data["text"].tolist())

    # load label encoder and model
    label_encoder = torch.load(f"../../models/xgboost/{dataset}/label_encoder.pt")
    model = torch.load(f"../../models/xgboost/{dataset}/model.pt")

    # run inference
    y_pred = model.predict(X)
    # add predictions to the DataFrame
    data['y_pred'] = y_pred

    # Encode the labels
    data["y_encoded"] = label_encoder.transform(data["y"])

    # Calculate metrics
    accuracy = accuracy_score(data["y_encoded"], data["y_pred"])
    precision = precision_score(data["y_encoded"], data["y_pred"], average="macro")
    recall = recall_score(data["y_encoded"], data["y_pred"], average="macro")
    f1 = f1_score(data["y_encoded"], data["y_pred"], average="macro")

    return {
        "model": "xgboost",
        "dataset": dataset,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [23]:
for d in datasets:
    evaluation_metrics.append(evaluate_xgboost(d))

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.77it/s]


KeyboardInterrupt: 

# Persist results

In [ ]:
pd.DataFrame(evaluation_metrics).to_csv("../../data/processed/evaluation_metrics.csv", index=False)